In [1]:
import pandas as pd
# pd.set_option('display.max_columns', 100)
from scipy.stats import mode
import geocoder
import time
import os
import sys
sys.path.insert(0, '/home/stang/user-profile/stats-core')
from utils import load_header_config

In [2]:
def truncate(f, n):
    '''Truncates/pads a float f to n decimal places without rounding'''
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

In [3]:
processed = set()
with open('res.txt', 'r') as inf:
    for line in inf:
        processed.add(line.split(',')[0])

In [4]:
base = '/gaei/gacrnd/data/ag_by_vin/'
files = sorted([f for f in os.listdir(base) if 'vin_L' in f])
header, configs = load_header_config('aglog_header.csv')
for i, f in enumerate(files):
    vin = f.split('.')[0].split('_')[1]
    
    if vin in processed:
        continue

    fname = base + f
    df = pd.read_csv(fname, header=None, names=header, index_col=False)
    df = df.iloc[:, 74:76].dropna()

    df.columns = ['lon', 'lat']
    df['lon'] = df['lon'].apply(lambda x: truncate(x, 2))
    df['lat'] = df['lat'].apply(lambda x: truncate(x, 2))
    df = df[(df['lat'] != '0.00') & (df['lon'] != '0.00')]
    df = df.iloc[-10000:, :]
    
    df['coord'] = df['lat'] + '+' + df['lon']
    
    lat, lon = df['coord'].mode()[0].split('+')
    with open('res.txt', 'a') as outf:
        outf.write('{},{},{}\n'.format(vin, lat, lon))
    print(i, vin, lat, lon)
    print('progress: {:.2f}%'.format(float(i)/len(files)*100))
    print('-----------------------')
print('done')

done


In [34]:
f = 'vin_LMGGN1S52G1001793.csv'
fname = base + f
df = pd.read_csv(fname, header=None, names=header, index_col=False)
df = df.iloc[:, [1,74,75]].dropna()
df.loc[:, 'TDATE'] = pd.to_datetime(df['TDATE'], unit='ms', utc=True).dt.tz_convert('Asia/Hong_Kong')
df = df.sort_values(by='TDATE')

In [35]:
df.head()

,TDATE,LON84,LAT84
0,2016-03-05 09:41:38+08:00,113.486670,23.042800
1,2016-03-05 09:41:48+08:00,113.486664,23.042875
2,2016-03-05 09:41:58+08:00,113.486550,23.042774
3,2016-03-05 09:42:09+08:00,113.486580,23.042744
5,2016-03-05 09:42:19+08:00,113.486590,23.042727


In [36]:
df.tail()

,TDATE,LON84,LAT84
19482,2017-06-10 16:25:39+08:00,121.629958,39.035472
19483,2017-06-10 16:25:49+08:00,121.630275,39.035824
19478,2017-06-10 16:26:00+08:00,121.630608,39.036183
19479,2017-06-10 16:26:10+08:00,121.630766,39.036205
19480,2017-06-10 16:26:20+08:00,121.630769,39.036208


In [37]:
df.columns = ['TDATE', 'lon', 'lat']
df['lon'] = df['lon'].apply(lambda x: truncate(x, 2))
df['lat'] = df['lat'].apply(lambda x: truncate(x, 2))
df = df[(df['lat'] != '0.00') & (df['lon'] != '0.00')]
df['coord'] = df['lat'] + '+' + df['lon']

In [44]:
df.iloc[-100:, 3].mode()[0].split('+')

['39.03', '121.62']

In [38]:
df.head()

,TDATE,lon,lat,coord
0,2016-03-05 09:41:38+08:00,113.48,23.04,23.04+113.48
1,2016-03-05 09:41:48+08:00,113.48,23.04,23.04+113.48
2,2016-03-05 09:41:58+08:00,113.48,23.04,23.04+113.48
3,2016-03-05 09:42:09+08:00,113.48,23.04,23.04+113.48
5,2016-03-05 09:42:19+08:00,113.48,23.04,23.04+113.48


In [39]:
df['coord'].value_counts()

37.55+121.37    6993
39.01+121.70    6271
31.26+121.34    2698
31.08+121.43     785
31.23+121.36     628
23.04+113.48     558
37.55+121.38     358
39.03+121.62     146
31.25+121.34      86
31.31+121.14      67
31.10+121.39      49
31.26+121.35      48
31.10+121.40      47
31.23+121.34      39
31.09+121.43      28
31.24+121.34      26
31.25+121.35      24
31.11+121.38      24
31.18+121.35      22
31.17+121.35      21
31.08+121.40      21
31.09+121.41      20
31.25+121.33      18
31.21+121.34      18
31.22+121.34      17
31.11+121.40      17
31.15+121.36      17
31.13+121.37      17
31.12+121.38      16
31.08+121.41      15
                ... 
31.25+121.18       5
39.01+121.66       5
39.01+121.63       5
39.01+121.65       5
31.25+121.28       5
31.25+121.29       5
31.25+121.31       5
39.02+121.68       5
31.25+121.21       4
31.25+121.20       4
31.28+121.13       4
39.02+121.67       4
37.56+121.38       4
31.25+121.19       4
31.25+121.17       4
31.25+121.25       4
31.25+121.23 

In [40]:
df[df['coord'] == '39.01+121.70']

,TDATE,lon,lat,coord
11114,2017-05-11 16:45:01+08:00,121.70,39.01,39.01+121.70
11115,2017-05-11 16:45:11+08:00,121.70,39.01,39.01+121.70
11116,2017-05-11 16:45:21+08:00,121.70,39.01,39.01+121.70
11117,2017-05-11 16:45:33+08:00,121.70,39.01,39.01+121.70
11118,2017-05-11 16:45:43+08:00,121.70,39.01,39.01+121.70
11119,2017-05-11 16:45:53+08:00,121.70,39.01,39.01+121.70
11120,2017-05-11 16:46:03+08:00,121.70,39.01,39.01+121.70
11121,2017-05-11 16:46:13+08:00,121.70,39.01,39.01+121.70
11122,2017-05-11 16:46:23+08:00,121.70,39.01,39.01+121.70
11123,2017-05-11 16:46:33+08:00,121.70,39.01,39.01+121.70


In [41]:
df[df['coord'] == '37.55+121.37']

,TDATE,lon,lat,coord
5756,2017-05-09 16:36:26+08:00,121.37,37.55,37.55+121.37
5757,2017-05-09 16:36:36+08:00,121.37,37.55,37.55+121.37
5758,2017-05-09 16:36:46+08:00,121.37,37.55,37.55+121.37
5759,2017-05-09 16:36:47+08:00,121.37,37.55,37.55+121.37
5760,2017-05-09 16:36:57+08:00,121.37,37.55,37.55+121.37
5761,2017-05-09 16:37:07+08:00,121.37,37.55,37.55+121.37
5762,2017-05-09 16:37:18+08:00,121.37,37.55,37.55+121.37
5763,2017-05-09 16:37:28+08:00,121.37,37.55,37.55+121.37
5764,2017-05-09 16:37:38+08:00,121.37,37.55,37.55+121.37
5765,2017-05-09 16:37:48+08:00,121.37,37.55,37.55+121.37


In [5]:
if not os.path.exists('veh_locs.csv'):
    d = {} # save address info, <vin, (city, province)>
    temp = pd.read_csv('final_veh_locs.csv', dtype='str')
    temp['coord']  = temp['lat'] + '+' + temp['lon']
    for key, city, province in zip(temp['coord'].tolist(), temp['city'].tolist(), temp['province'].tolist()):
        if key not in d:
            d[key] = (city, province)
    print(len(d))

    df1 = pd.read_csv('res.txt', dtype='str')
    df1.columns = ['vin', 'lat', 'lon']
    lats = df1['lat'].tolist()
    lons = df1['lon'].tolist()

    print('reverse geocoding coords..')
    for lat, lon in zip(lats, lons):
        key = lat + '+' + lon
        if key not in d:
            g = geocoder.gaode([float(lat), float(lon)], method='reverse', key='0716e5809437f14e3dd0793a5c6d2b13')
            while not g.ok:
                print('retrying..')
                time.sleep(5)
                g = geocoder.gaode([float(lat), float(lon)], method='reverse', key='0716e5809437f14e3dd0793a5c6d2b13')
            d[key] = (g.city, g.state)
    print('done.')

    fields = ['city', 'province']
    df1[fields] = df1.apply(lambda row: pd.Series(d[row['lat']+'+'+row['lon']]), axis=1)
    df1 = df1.sort_values(by='vin')
    df1 = df1.loc[:, ['vin', 'lat', 'lon', 'city', 'province']]
    df1.to_csv('veh_locs.csv', encoding='utf-8')

    print('done.')